# Defining AKS Resources with Terraform

In this lesson, we'll look in more detail at Azure Kubernetes Service (AKS) and Terraform, focusing on how to combine these powerful technologies to efficiently manage and orchestrate Kubernetes clusters on Azure.

## Key Objectives

- **Setting Up the Development Environment**: We'll start by preparing your development environment, ensuring that you have the necessary tools and access to Azure

- **Defining AKS Resources**: You'll learn how to define AKS resources in Terraform, including the creation of Azure Resource Groups and the configuration of AKS clusters to meet your specific requirements

- **Authentication and Deployment**: We'll cover the authentication process using the Azure CLI and demonstrate how to deploy your Terraform configuration to create AKS clusters seamlessly

- **Accessing the AKS Cluster**: Once your AKS cluster is up and running, we'll show you how to access it using the Kubernetes command-line tool, `kubectl`

- **Maintenance and Cleanup**: Managing your AKS cluster doesn't end with its creation. We'll explore various maintenance tasks, including scaling, backup, and upgrades. Additionally, you'll learn how to gracefully clean up and delete resources when they are no longer needed.

## Setting up the Development Environment

Before we start defining Azure Kubernetes Service (AKS) resources with Terraform, we need to ensure that your development environment is properly configured.

To create a robust and fully functional AKS cluster, we need to consider various components that make up the infrastructure. These components include:

- **Virtual Network (VNet)**: The networking foundation for your AKS cluster, which includes subnets for worker nodes, control plane nodes, and any additional network resources
- **Subnets**: Specific network segments within the VNet, including those for worker nodes and control plane nodes
- **Network Security Groups (NSGs)**: Security rules to control inbound and outbound traffic to resources in the subnets

These components are essential for configuring the network infrastructure to support the AKS cluster. Thus, this Terraform project will not only need to contain a module for defining the AKS cluster resource itself, but an additional networking module that will focus on creating the VNet, subnets, and NSGs required for your AKS environment.

Start by creating a dedicated directory for your Terraform project. This directory will contain all your Terraform configuration files.

```shell
mkdir aks-terraform
cd aks-terraform
```

Inside this directory, create two subdirectories: `networking-module` and `aks-cluster-module` to represent the networking services and the AKS cluster configuration, respectively.

```shell
mkdir networking-module
mkdir aks-cluster-module
```

By creating these directories, we are preparing to organize our Terraform modules in a structured manner. Now let's move on to the next sections, where we will define the necessary modules in detail and incorporate them into the main Terraform configuration.

## Networking Services Module Configuration

In this section, we will create the Terraform configuration files for the networking module, which provisions the networking components required to support your AKS cluster. The networking module includes the creation of a VNet, subnets, and NSG rules. These resources are important for secure access to your AKS cluster.

Start by navigating to the `networking-module` directory, where you will define the networking resources. Inside this directory, create the necessary Terraform configuration files:

- `main.tf`: This file defines the networking resources and NSG rules
- `variables.tf`: Define input variables 
- `outputs.tf` (optional): Define output variables 

### Define Input Variables (`variables.tf`)

In the `variables.tf` file, you can define input variables that allow you to customize your networking module's behavior. Input variables make your configuration more flexible and reusable.

Here's an example of how you can define input variables:

```hcl
# networking-module/variables.tf

variable "resource_group_name" {
  ...
}

variable "location" {
  ...
}

variable "vnet_address_space" {
  description = "Address space for the Virtual Network (VNet)."
  type        = list(string)
  default     = ["10.0.0.0/16"]
}

# Define more variables as needed...

```

The `resource_group_name` variable defines the name of the Azure Resource Group where the networking resources will be created. The `location` variable specifies the Azure region where networking resources, such as the VNet, will be created. The `vnet_address_space` variable defines the address space for the VNet. It is a list of strings representing IP address ranges in CIDR notation. 

These variables allow users to customize the name of the resource group, the Azure region, and the VNet address space when using the networking module. Users can override the default values by providing their desired values in their Terraform configuration.

### NSG Rules for Accessing AKS

To be able to access your AKS cluster once it's provisioned from your local computer using `kubectl`, you will need to ensure that it has the necessary NSG rules in place. Below are the key NSG rules to consider adding:

- `kube-apiserver` Port (TCP/443): The `kube-apiserver` serves the Kubernetes API. We create an NSG rule to allow inbound traffic to port 443 (HTTPS) on the `kube-apiserver`. This port is used for secure communication with the API server.

- SSH Port (TCP/22) (Optional): If you need SSH access to the nodes in your AKS cluster for troubleshooting or administrative purposes, you can create an NSG rule to allow inbound traffic to port 22 (SSH). This rule is optional and should only be added if necessary.

### Define Networking Resources and NSG Rules

In the `main.tf` file within the `networking-module` directory, we will define the networking resources and NSG rules needed for secure access to the AKS cluster.

```hcl
# networking-module/main.tf

# Create the Azure Resource Group for networking resources
resource "azurerm_resource_group" "networking" {
  name     = var.resource_group_name
  location = var.location
}

# Define the Virtual Network (VNet) for the AKS cluster
resource "azurerm_virtual_network" "aks_vnet" {
  name                = "<your_vnet_name>"
  address_space       = var.vnet_address_space
  location            = azurerm_resource_group.networking.location
  resource_group_name = azurerm_resource_group.networking.name
}

# Define subnets within the VNet for control plane and worker nodes
resource "azurerm_subnet" "control_plane_subnet" {
  name                 = "<your_control_plane_subnet_name>"
  resource_group_name  = azurerm_resource_group.networking.name
  virtual_network_name = azurerm_virtual_network.aks_vnet.name
  address_prefixes     = ["10.0.1.0/24"]
}

resource "azurerm_subnet" "worker_node_subnet" {
  name                 = "<your_worker_node_subnet_name>"
  resource_group_name  = azurerm_resource_group.networking.name
  virtual_network_name = azurerm_virtual_network.aks_vnet.name
  address_prefixes     = ["10.0.2.0/24"]
}

# Define Network Security Group (NSG) for the AKS subnet
resource "azurerm_network_security_group" "aks_nsg" {
  name                = "<your_nsg_name>"
  location            = azurerm_resource_group.networking.location
  resource_group_name = azurerm_resource_group.networking.name
}

# Allow inbound traffic to kube-apiserver (TCP/443) from your public IP address
resource "azurerm_network_security_rule" "kube_apiserver" {
  name                        = ""<your_nsg_rule1_name>"
  priority                    = 1001
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "443"
  source_address_prefix       = "YOUR_PUBLIC_IP_ADDRESS"  # Replace with your public IP or IP range
  destination_address_prefix  = "*"
  resource_group_name         = azurerm_resource_group.networking.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
}

# Allow inbound traffic for SSH (TCP/22) - Optional
resource "azurerm_network_security_rule" "ssh" {
  name                        = "<your_nsg_rule2_name>"
  priority                    = 1002
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "22"
  source_address_prefix       = "YOUR_PUBLIC_IP_ADDRESS"  # Replace with your public IP or IP range
  destination_address_prefix  = "*"
  resource_group_name         = azurerm_resource_group.networking.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
}
```

> Make sure to replace the names in the resource blocks with your desired name for each resource. For example replace `"<your_nsg_name>"` with `"aks-nsg`.

In the example above:

- The `azurerm_resource_group` block defines the creation of an Azure Resource Group, which is a logical container for grouping related Azure resources. The `name` and `location` parameters are set based on input variables (`var.resource_group_name` and `var.location`) to specify the name and Azure region for the resource group.

- The `azurerm_virtual_network` block defines the creation of a Virtual Network (VNet) where your AKS cluster will reside. The `address_space` parameter is set based on the `var.vnet_address_space` input variable, which defines the address space for the VNet in CIDR notation.

- The two `azurerm_subnet` subnets are defined within the VNet: `control_plane_subnet` and `worker_node_subnet`. These subnets are used for specific purposes within your AKS cluster. The `control_plane_subnet` is a subnet created within the Azure VNet that is dedicated to hosting the control plane components of an AKS cluster. The `worker_node_subnet` is another subnet created within the Azure VNet, and it serves as the network space for hosting the worker nodes of the AKS cluster.

- The `azurerm_network_security_group` block defines the creation of an NSG named `aks-nsg`, which will be associated with the AKS subnet to control inbound and outbound traffic

- The two `azurerm_network_security_rule` NSG rules are defined for inbound traffic:

  - `kube_apiserver`: This rule allows inbound traffic on TCP port 443 (HTTPS) from a specified public IP address (`YOUR_PUBLIC_IP_ADDRESS`). This is essential for secure communication with the `kube-apiserver` of your AKS cluster.

  - `ssh` (Optional): This rule allows inbound traffic on TCP port 22 (SSH) from a specified public IP address (`YOUR_PUBLIC_IP_ADDRESS`). It's optional and should only be added if SSH access to AKS nodes is required for administrative purposes.

Overall, this configuration file sets up the networking infrastructure for your AKS cluster. It creates a resource group, a Virtual Network with subnets, and defines NSG rules to control inbound traffic. The NSG rules allow secure access to the Kubernetes API server (`kube-apiserver`) and optionally provide SSH access for administrative tasks.

> Remember to replace `YOUR_PUBLIC_IP_ADDRESS` with your actual public IP address or IP range in the NSG rules to restrict access to only authorized IPs. You can run `curl ipinfo.io/ip` to find your IP address.

## Define Output Variables (`outputs.tf`)

In the `outputs.tf` file, you can define output variables that provide information about the created networking resources. Output variables are useful for retrieving information from your modules and making it available for use in other parts of your Terraform configuration.

Here's an example of how you can define output variables:

```hcl
# networking-module/outputs.tf

output "vnet_id" {
  description = "ID of the Virtual Network (VNet)."
  value       = <vnet-resource-type>.<vnet-resource-identifier>.id
}

output "control_plane_subnet_id" {
  description = "ID of the control plane subnet."
  value       = <subnet-resource-type>.<control-plane-subnet-identifier>.id
}

output "worker_node_subnet_id" {
  description = "ID of the worker node subnet."
  value       = <subnet-resource-type>.<worker-node-subnet-identifier>.id
}

output "resource_group_name" {
  description = "Name of the Azure Resource Group for networking resources."
  value       = <resource-group-resource-type>.<resource-group-resource-identifier>.name
}

# Define more output variables as needed...
output "aks_nsg_id" {
  description = "ID of the Network Security Group (NSG) for AKS."
  value       = <nsg-resource-type>.<nsg-resource-identifier>.id
}
```

> Within the `value` argument of each `output` block you will have to replace the `<resource-type>` and `<resource-identifier>` with the correct resource type and identifier for each resource. You can find these values in the previously created `main.tf` where you provisioned your resources.

Let's look at an example, for the VNet resource:

```hcl
# Define the Virtual Network (VNet) for the AKS cluster
resource "azurerm_virtual_network" "aks_vnet" {
  name                = "<your_vnet_name>"
  address_space       = var.vnet_address_space
  location            = azurerm_resource_group.networking.location
  resource_group_name = azurerm_resource_group.networking.name
}
```

The `output` block should look like this:

```hcl
output "vnet_id" {
  description = "ID of the Virtual Network (VNet)."
  value       = azurerm_virtual_network.aks_vnet.id
}
```
In this case `<resource-type>` corresponds to `azurerm_virtual_network` and `<resource-identifier>` corresponds to `aks_vnet` as we can see from the VNet `resource` block.

The `vnet_id` output variable provides the ID of the VNet created by the networking module. This output variable can then be used in the AKS cluster module to specify the VNet in which the AKS cluster should be created. 

The `control_plane_subnet_id` output variables provides the ID of the control plane subnet within the VNet. The control plane subnet is where the control plane components of the AKS cluster, such as the `kube-apiserver`, are deployed. You can use this output variable to configure the AKS cluster to use the specified subnet for its control plane. 

The `worker_node_subnet_id` output variables provides the ID of the worker node subnet within the VNet. The worker node subnet is where the AKS cluster's worker nodes are deployed. You can use this output variable to configure the AKS cluster to use the specified subnets for its worker nodes.

The `resource_group_name` output variable will be used here as an output variable and will be used as an input variable in the AKS cluster module. This will allow you to pass data and share information between the modules in a structured and controlled manner.

### Review and Apply Configuration

Run `terraform init` to set up the Terraform workspace and download the necessary provider plugins and modules. 

We will proceed with defining the AKS cluster module's configuration in the next section, and then we will apply both modules together, from the main project directory.

## Define AKS Cluster Module Resources

In this section, we will define the Terraform configuration for creating an AKS cluster. Before we start defining the AKS cluster configuration, navigate to the module directory for your AKS resources `aks-cluster-module`. 

Inside the AKS module directory, create the following files:

- `variables.tf`: This file will define the input variables that allow customization of the AKS cluster
- `outputs.tf`: In this file, you can specify any output values that you want to retrieve after provisioning the AKS cluster
- `main.tf`: The main configuration file where we will define the actual AKS cluster resources

### Define Input Variables (`variables.tf`) 

To create a flexible and customizable AKS cluster configuration, we will define input variables that can be customized when using this module. Here are the input variables for this module:

```hcl
# aks-cluster/variables.tf

variable "aks_cluster_name" {
  ...
}

variable "cluster_location" {
  ...
}

variable "dns_prefix" {
  ...
}

variable "kubernetes_version" {
  ...
}

variable "service_principal_client_id" {
  ...
}

variable "service_principal_client_secret" {
  ...
}

# Input variables from the networking module
variable "resource_group_name" {
  ...
}

variable "vnet_id" {
  ...
}

...
```
> Inside the `variables.tf` file of this module, you should make sure to include all the output variables from the networking module. As an example `resource_group_name` and `vnet_id` have already been included. Make sure to add the rest of them including `control_plane_subnet_id`, `worker_node_subnet_id` and `aks_nsg_id`.

Here's an explanation of the new input variables defined in the AKS cluster module configuration, excluding the ones that are output variables from the networking module. Later, you will use the output values from networking module as input variables in the AKS cluster module. This allows you to pass data and share information between modules in a structured and controlled manner.

- The `aks_cluster_name` variable specifies the name of the AKS cluster that will be created
- The `cluster_location` defines the Azure region where the AKS cluster will be created
- The `dn_prefix` sets the DNS prefix for the AKS cluster, which is used to create a unique DNS name for the cluster
- The `kubernetes_version` specifies the version of Kubernetes to be used for the AKS cluster
- The `service_principal_client_id` is the Client ID of the service principal used for authenticating and managing the AKS cluster
- The `service_principal_client_secret` specifies the Client Secret associated with the service principal used for AKS cluster authentication

We will use these input variables to configure and create the AKS cluster resources in the `main.tf` file.

### Define AKS Cluster Module

In the `main.tf`, you can now define the configuration for the AKS cluster module.

```hcl
# aks-cluster/main.tf
# Create the AKS cluster
resource "azurerm_kubernetes_cluster" "aks_cluster" {
  name                = var.aks_cluster_name
  location            = var.cluster_location
  resource_group_name = var.resource_group_name
  dns_prefix          = var.dns_prefix
  kubernetes_version  = var.kubernetes_version

  default_node_pool {
    name       = "default"
    node_count = 1
    vm_size    = "Standard_DS2_v2"
    enable_auto_scaling = true
    min_count = 1
    max_count = 3
  }

  service_principal {
    client_id     = var.service_principal_client_id
    client_secret = var.service_principal_client_secret
  }
}

```
Let's break down the configuration block for creating the AKS cluster:

```# Create the AKS cluster
resource "azurerm_kubernetes_cluster" "aks_cluster" {
  name                = var.aks_cluster_name
  location            = var.cluster_location
  resource_group_name = var.resource_group_name
  dns_prefix          = var.dns_prefix
  kubernetes_version  = var.kubernetes_version
```

This block specifies the information we already explained in detail when defining the `variables.tf` for this module.

```hcl
  default_node_pool {
    name       = "default"
    node_count = 1
    vm_size    = "Standard_DS2_v2"
    enable_auto_scaling = true
    min_count = 1
    max_count = 3
  }
```

The `default_node_pool` block defines the default node pool for the cluster:

- `name`: Specifies the name of the default node pool, which is `default` in this case
- `node_count`: Sets the initial number of nodes in the node pool to 1
- `vm_size`: Specifies the virtual machine size for the nodes in the pool, which is `Standard_DS2_v2` in this example
- `enable_auto_scaling`: Enables auto-scaling for the node pool
- `min_count`: Sets the minimum number of nodes to 1
- `max_count`: Sets the maximum number of nodes to 3

```hcl
  service_principal {
    client_id     = var.service_principal_client_id
    client_secret = var.service_principal_client_secret
  }
}
```

Finally, the `service_principal` blocks provides the authentication details for the AKS cluster. These are the same credentials you use when defining an Azure provider in your configuration. You should have access to this if you have provisioned a Service Principal in the previous lessons.

### Define Output Variables (`outputs.tf`) 

In the `outputs.tf` define the output variables for the AKS cluster module:

```hcl
output "aks_cluster_name" {
  description = "Name of the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.name
}

output "aks_cluster_id" {
  description = "ID of the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.id
}

output "aks_kubeconfig" {
  description = "Kubeconfig file for accessing the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.kube_config_raw
}
```
> Within the `value` argument of each `output` block you will have to replace the `<resource-type>` and `<resource-identifier>` with the correct resource type and identifier for each resource. You can find these values in the previously created `main.tf` where you provisioned your resources.

These output variables provide information about the AKS cluster, including its name, ID, and the `kubeconfig` file for accessing the cluster using `kubectl`.

### Review and Apply Configuration

Run `terraform init` to set up the Terraform workspace and download the necessary provider plugins and modules. 

In the next section we will apply both modules together, from the main project directory.

## Define the Main Configuration

In this section, we will define the main configuration for your project, which will utilize the networking module and the AKS cluster module to provision the necessary Azure resources.

Begin by navigating to the project directory `aks-terraform`. Here we will begin defining your project's main configuration file, `main.tf`, by defining the Azure provider block for authentication. This block should be included at the beginning of the configuration file. 

```hcl
terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "=3.0.0"
    }
  }
}

provider "azurerm" {
  features {}
  client_id       = var.client_id
  client_secret   = var.client_secret
  subscription_id = "your-subscription-id"
  tenant_id       = "your-tenant-id"
}
```
This provider block specifies that you will be using Azure as your cloud provider. It uses variables for the `client_id` and `client_secret`, which we will store in environment variables to avoid hardcoding them. First we need to add them to the main configuration's `variables.tf` as follows:

```hcl
#variables.tf

variable "client_id" {
  description = "Access key for the provider"
  type        = string
  sensitive   = true
}

variable "client_secret" {
  description = "Secret key for the provider"
  type        = string
  sensitive   = true
}
```

To use these variables securely, you will then need to open your `.zshrc` in a CLI text editor and add the following lines:

```bash
export TF_VAR_client_id=<your-client-id-here>
export TF_VAR_client_secret=<your-client-secret-here>
```

The `TF_VAR_` prefix will allow Terraform to find these variables automatically and assign them to the ones you defined in `variables.tf`. Once you have added the variable definitions, save and exit your CLI text editor, and run the following to reinitialise your dotfile:

```bash
source ~/.zshrc
```

>IMPORTANT: Even if you use environment variables to hide your credentials during `terraform plan` and `terraform apply`, the secrets will still be present in your statefile. DO NOT upload your statefile to Github! Make sure to add it to your `.gitignore` to avoid any risk of this.


### Use the Networking Module

Now, let's use the networking module you created to provision the networking resources needed for your AKS cluster. In your `main.tf`, include the following code:

```hcl
# main.tf

module "networking" {
  source = "./networking-module"

  # Input variables for the networking module
  resource_group_name = "networking-rg"
  location           = "UK South"
  vnet_address_space = ["10.0.0.0/16"]

  # Define more input variables as needed...
}
```
This code imports the networking module and provides values for the required input variables.

### Use the AKS Cluster Module

Next, utilize the AKS cluster module to define and provision your AKS cluster. Add the following code to your `main.tf`:

```hcl
# main.tf

module "aks_cluster" {
  source = "./aks-cluster-module"

  # Input variables for the AKS cluster module
  aks_cluster_name           = "terraform-aks-cluster"
  cluster_location           = "UK South"
  dns_prefix                 = "myaks-project"
  kubernetes_version         = "1.26.6"  # Adjust the version as needed
  service_principal_client_id = var.your-service-principal-client-id
  service_principal_client_secret = var.your-service-principal-client-secret

  # Input variables referencing outputs from the networking module
  resource_group_name         = module.networking.resource_group_name
  vnet_id                     = module.networking.vnet_id
  control_plane_subnet_id     = module.networking.control_plane_subnet_id
  worker_node_subnet_id       = module.networking.worker_node_subnet_id
  aks_nsg_id                  = module.networking.aks_nsg_id

  # Define more input variables as needed...
}
```

In this code, you import the AKS cluster module and provide values for the input variables required by the module. Note you will once again need to add input variables to `variables.tf`, and environment variables to `.zshrc`, for your service principal ID and secret.

### Review and Apply Configuration

After defining the main configuration file, you can now review the changes and plan the deployment:

```shell
terraform init
terraform plan
```

If the plan looks satisfactory (it should plan to deploy 8 different resources), you can apply the configuration to create the networking resources and the AKS cluster using `terraform apply`. With these networking configurations and the AKS cluster in place, your infrastructure will be set up according to your specifications. Be mindful that this might take a couple of minutes. Congratulations on deploying your first AKS cluster using Terraform!

## Accessing the AKS Cluster

Once you've deployed your AKS cluster, you'll need to access it to manage and deploy applications. In this section, we'll guide you on how to retrieve the `kubeconfig` and use `kubectl` to interact with the cluster.

### Retrieve `kubeconfig`

The `kubeconfig` is a configuration file that allows you to connect to your AKS cluster securely. To retrieve the `kubeconfig`, follow these steps:

```shell
# Use the Azure CLI to get the AKS cluster credentials
az aks get-credentials --resource-group <your-resource-group> --name <your-aks-cluster-name>
```

Replace `<your-resource-group>` and `<your-aks-cluster-name>` with the appropriate values for your AKS cluster.

The above command will fetch the `kubeconfig` and automatically merge it into your local `~/.kube/config` file, which is the default location for `kubeconfig`. Ensure that you have Azure CLI installed and configured with the necessary permissions.

<p align=center><img src=images/kubeFile.png width=600 height=350></p>

After retrieving the `kubeconfig`, you can start using `kubectl` to interact with your AKS cluster. For example check the status of your nodes you can run `kubectl get nodes`.

With these steps, you can easily access and manage your AKS cluster using `kubectl`, making it a powerful tool for deploying and managing Kubernetes workloads.

## Clean Up Resources

When you no longer need your AKS cluster and associated resources, it's crucial to clean up to avoid unnecessary costs. Use Terraform's `terraform destroy` command to deprovision AKS and related resources while maintaining infrastructure as code (IaC) principles.

Before performing any destructive actions, double-check that you have backups of critical data and configurations.

## Conclusion

In this notebook, we explored the process of provisioning, configuring, and managing Azure Kubernetes Service (AKS) clusters using Terraform. By following Infrastructure as Code (IaC) principles, modularization, and best practices, you learned how to efficiently create and maintain AKS clusters to run containerized applications at scale.